### Identity Access Management (IAM)
##### External IAM Access Analyzers for Organization
IAM Access Analyzer must have Delegated Administrator assigned to the **Audit** account in a Control Tower environment.

In [ ]:
import botoplus.organization as _org

selected = _org.selected()
ec2 = selected.client('ec2')
regions = ec2.describe_regions()

for region in regions['Regions']:

    client = selected.client('accessanalyzer', region_name = region['RegionName'])
    paginator = client.get_paginator('list_analyzers')
    pages = paginator.paginate(
        type = 'ORGANIZATION'
    )

    exists = []

    for page in pages:
        for item in page['analyzers']:
            exists.append(item['name'])

    if 'organization' in exists:

        print(' - '+region['RegionName']+' organization: EXISTS')

    else:

        accessanalyzer = selected.client('accessanalyzer', region_name = region['RegionName'])
        
        accessanalyzer.create_analyzer(
            analyzerName = 'organization',
            type = 'ORGANIZATION'
        )

        print(' - '+region['RegionName']+' organization: CREATED')

##### External IAM Access Analyzers for Accounts

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:

        try:

            client = sessions.client('accessanalyzer', region_name = region['RegionName'])
            paginator = client.get_paginator('list_analyzers')
            pages = paginator.paginate(
                type = 'ACCOUNT'
            )

            exists = []

            for page in pages:
                for item in page['analyzers']:
                    exists.append(item['name'])

            if 'awsaccount' in exists:

                print(' - '+region['RegionName']+' awsaccount: EXISTS')

            else:

                accessanalyzer = sessions.client('accessanalyzer', region_name = region['RegionName'])
        
                accessanalyzer.create_analyzer(
                    analyzerName = 'awsaccount',
                    type = 'ACCOUNT'
                )

                print(' - '+region['RegionName']+' awsaccount: CREATED')

        except:

            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass